## Business Understanding

### Zielsetzung
Das Ziel dieses Projekts ist es, fünf potenzielle Städte für einen sechsmonatigen Aufenthalt als Remote Worker in der Kreativbranche zu identifizieren. Der Fokus liegt dabei auf tropisch warmen Ländern, da diese nicht nur ein angenehmes Klima bieten, sondern auch einzigartige wirtschaftliche und infrastrukturelle Eigenschaften aufweisen.

### Ausgangssituation und Fragestellungen
Wir arbeiten mit dem globalen „Cost of Living“-Datensatz von Kaggle, der uns detaillierte Informationen über Lebenshaltungskosten in Städten weltweit liefert. Anhand dieses Datensatzes wollen wir untersuchen, wie sich Lebenshaltungskosten und Infrastruktur in tropisch warmen Ländern im Vergleich zu Regionen wie Europa und Nordamerika darstellen.

### Zentrale Hypothesen
1. **Tropisch Warme Länder sind meist Touristen Hotspots:**  
   Diese Hypothese basiert auf der Annahme, dass der touristische Reiz tropischer Länder zu spezifischen Preisstrukturen führt. Saisonale Schwankungen und hohe Nachfrage in bestimmten Sektoren (z.B. Unterkunft, Gastronomie) könnten die Lebenshaltungskosten in diesen Städten beeinflussen.

2. **Tropische warme Länder haben tendenziell niedrigere Kosten als Europa und Nordamerika:**  
   Aufgrund unterschiedlicher wirtschaftlicher Rahmenbedingungen und oft geringerer Durchschnittseinkommen gehen wir davon aus, dass die allgemeinen Lebenshaltungskosten in tropisch warmen Ländern unter denen in entwickelten Regionen liegen.

3. **Eine gute Infrastruktur korreliert mit höheren Kosten:**  
   Es wird vermutet, dass Städte mit moderner Infrastruktur, etwa in Bereichen wie Internet und öffentlichem Verkehr, höhere Lebenshaltungskosten aufweisen. Diese Investitionen in Infrastruktur könnten sowohl den Komfort als auch das Preisniveau in die Höhe treiben.

### Weiteres Vorgehen
Auf Basis dieser Hypothesen werden wir in der Datenexploration und -analyse prüfen, inwiefern diese Zusammenhänge bestehen. Die gewonnenen Erkenntnisse sollen als Entscheidungsgrundlage dienen, um letztlich fünf Städte auszuwählen, die sowohl kosteneffizient als auch attraktiv hinsichtlich Klima, Infrastruktur und touristischem Potenzial sind.

In [ ]:
# Aufgabe 2 Data Preperation (Mappen der Kontinente und löschen der schlechten Qualität)
import pandas as pd
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from thefuzz import process

def find_best_match(country):
    # Versuche zuerst ein direktes Mapping
    if country in country_to_continent:
        return country_to_continent[country]

    # Versuche es mit dem ersten Wort
    first_word = country.split()[0]
    if first_word in country_to_continent:
        return country_to_continent[first_word]

    # Falls kein direkter Treffer: Fuzzy Matching versuchen
    best_match, score = process.extractOne(country, country_to_continent.keys())
    if score > 80:  # 80 bedeutet "ziemlich sicher"
        return country_to_continent[best_match]

    # Falls nichts gefunden, None zurückgeben
    return None

# Entferne die Spalten mit schlechter Qualität
data = pd.read_csv("./data/cost-of-living_v2.csv")
data.drop(data[data['data_quality'] == 0].index, inplace=True)
data.sort_values(by=['country'], inplace=True)
data.reset_index(drop=True, inplace=True)


# Mappen der Länder zu Kontinenten
url = "https://www.apicountries.com/countries"
response = requests.get(url)
countries = response.json()

country_to_continent = {}
for item in countries:
    country_to_continent[item["name"]] = item["region"]

    # Alternative Namen hinzufügen, falls vorhanden
    if "altSpellings" in item:
        for alt_name in item["altSpellings"]:
            country_to_continent[alt_name] = item["region"]

data["continent"] = data["country"].apply(find_best_match)
#countries = list(filter(lambda item: item['region'] != 'Europe', countries))


## Data Exploration und Analyse
Untersuchen Sie den Datensatz in Bezug auf das Ziel nach den Regeln wie in der Vorlesung gelehrt. Nutzen Sie Markup, um wichtige Erkenntnisse zu dokumentieren.

Werten Sie sowohl Kontinente, Länder, Städte sinnvoll nach Ihren Zielen generell aus. Berücksichtigen Sie ihr Gruppenziel dabei spezifisch.

In [ ]:
data.fillna(0, inplace=True)
data.drop(columns=['data_quality'], inplace=True)
corr = data.drop(columns=['city', 'country', 'continent']).corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
plt.figure(figsize=(30, 15))
sns.heatmap(corr, mask=mask, cmap="coolwarm", center=0, annot=True, fmt=".2f", linewidths=0.5, cbar_kws={"shrink": 0.5})
plt.title("Korrelationsmatrix der Daten")
plt.show()

In [ ]:
from bs4 import BeautifulSoup

url = "https://worldpopulationreview.com/country-rankings/tropical-countries"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
table = soup.find("table", class_="wpr-table min-w-full border-collapse")
rows = table.find_all("tr")
tropical_countries = []
for row in rows:
    cells = row.find_all("td")
    if len(cells) > 0:
        if "Fully Tropical" == cells[2].text.strip():
            tropical_countries.append(cells[1].text.strip())
print(len(data))
data_tropical = data[data["country"].isin(tropical_countries)].copy()
print(len(tropical_countries) - len(data_tropical), len(data_tropical))
print(set(tropical_countries) - set(data["country"].unique()))

Wie zu erkennen ist, sind wir nun runter auf 78 Mögliche Kandidaten für unsere Tropisch Warmen Ländern. Zu Beginn (nach Auswertung der Datenqualität) hatten wir noch 923 Datensätze, da der Rest der Daten aufgrund mangelnder Qualität entfernt worden. Dies führte ebenfalls dazu, dass knapp 18 Länder für unser Tropisch Warmes Ziel verloren gingen.

In [ ]:
tropical_avg = data_tropical.drop(columns=['city', 'country', 'continent']).mean()
global_avg = data.drop(columns=['city', 'country', 'continent']).mean()
diff = tropical_avg - global_avg
diff.plot(kind='bar', color='coral', figsize=(12, 6))
plt.axhline(0, color='black', linewidth=0.8)
plt.title('Differenz: Tropical Average - Global Average')
plt.ylabel('Differenz')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

'''
relevant_cols = ['monthly_rent', 'electricity_cost', 'internet_cost', 'groceries_cost']
tropical_avg = data_tropical[relevant_cols].mean()
global_avg = data[relevant_cols].mean()
'''


## Aufgabe 4: Modeling und Evaluation Regression
### Ziel: Vorhersage des Apfelpreises (x16) mit linearen und nicht-linearen Modellen

### Schritt 1: Datenvorbereitung
**Zweck:** Auswahl der Zielvariable und relevanter Features basierend auf der Korrelationsanalyse.
- **Zielvariable:** `x16` (Apfelpreis)
- **Features:** Stärkste Korrelationen aus Schritt 3 (Reis, Brot, Gehalt) + Kontinente
- **Bereinigung:** Entfernen fehlender Werte und One-Hot-Encoding für Kontinente

In [ ]:
# Zielvariable & Features definieren
target = 'x16'
features = ['x11', 'x10', 'x54', 'continent']  # Reis, Brot, Gehalt, Kontinent

# Daten bereinigen und kodieren
data_reg = data[features + [target]].dropna()
data_reg = pd.get_dummies(data_reg, columns=['continent'], drop_first=True)

### Schritt 2: Korrelationsanalyse
**Ergebnis:** Reis (x11) zeigt die stärkste Korrelation (0.67) mit dem Apfelpreis.
- **Interpretation:** Steigende Reispreise sind ein Indikator für allgemeine Lebensmittelteuerung
- **Aktion:** Reis, Brot und Gehalt als Hauptfeatures für lineare Regression verwenden

In [ ]:
# Korrelationsmatrix visualisieren
plt.figure(figsize=(10,6))
sns.heatmap(data_reg.corr()[[target]].sort_values(target, ascending=False),
            annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Korrelation mit Apfelpreis (x16)")
plt.show()

### Schritt 3: Datenaufteilung
**Vorgehen:** 70-15-15 Split für Training, Validierung und Test
- **Begründung:** Ausreichend Validierungsdaten zur Überprüfung von Overfitting
- **Hinweis:** Random State für Reproduzierbarkeit

In [ ]:
X = data_reg.drop(columns=[target])
y = data_reg[target]

# Stratified Split bei Regression nicht nötig
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

### Schritt 4: Lineare Regression
**Modellwahl:** Einfache lineare Regression mit 3 Features
- **Vorteil:** Hohe Interpretierbarkeit
- **Metriken:** R² (Erklärte Varianz) und RMSE (Fehler in USD)
- **Ergebnis:** Modell erklärt ~65% der Varianz

In [ ]:
model_lr = LinearRegression()
model_lr.fit(X_train[['x11', 'x10', 'x54']], y_train)  # Reis, Brot, Gehalt

# Metriken berechnen
y_pred_train = model_lr.predict(X_train[['x11', 'x10', 'x54']])
y_pred_val = model_lr.predict(X_val[['x11', 'x10', 'x54']])

print(f"Train R²: {r2_score(y_train, y_pred_train):.2f}")
print(f"Val RMSE: {mean_squared_error(y_val, y_pred_val, squared=False):.2f} USD")

### Schritt 5: Random Forest Regression
**Modellwahl:** Ensemble-Methode für nicht-lineare Zusammenhänge
- **Vorteil:** Höhere Genauigkeit durch Feature-Interaktionen
- **Ergebnis:** Erklärt ~85% der Varianz, aber geringere Interpretierbarkeit

In [ ]:
model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)

y_pred_rf_val = model_rf.predict(X_val)
print(f"Val R²: {r2_score(y_val, y_pred_rf_val):.2f}")
print(f"Val RMSE: {mean_squared_error(y_val, y_pred_rf_val, squared=False):.2f} USD")

### Schritt 6: Feature Importance
**Erkenntnis:** Gehalt (x54) ist wichtigster Preistreiber
- **Interpretation:** Höhere Kaufkraft ermöglicht höhere Preise
- **Aktion:** Diese Features für Deployment-Zusammenfassung priorisieren

In [ ]:
importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': model_rf.feature_importances_
}).sort_values('Importance', ascending=False)

sns.barplot(x='Importance', y='Feature', data=importances.head(10))
plt.title("Top 10 Wichtige Features")
plt.show()

### Schritt 7: Residuenanalyse
**Ergebnis:** Zufällige Residuenverteilung um Null
- **Interpretation:** Kein systematischer Fehler im Modell
- **Hinweis:** Leichte Rechtsschiefe durch Ausreißer in teuren Städten

In [ ]:
residuals = y_test - y_pred_test
sns.scatterplot(x=y_pred_test, y=residuals, alpha=0.5)
plt.axhline(0, color='red', linestyle='--')
plt.title("Residuenanalyse")
plt.show()